### Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from pathlib import Path
import os

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf

### Directory to read data

In [ ]:
##################################################################################################################
# Simulation paths

##########################
# CFI simulations


directory_path = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_00.1m_att_1e-3'
ncell = (1,1,1) # number of cells in each direction
domain = (1e5, 1e5, 1e5) # cm
lowlimfit = 20 # data point to start fit to compute omega_im
uplimfit  = 60 # data point to end fit to compute omega_im

directory_path = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_01.0m_att_1e-4'
ncell = (1,1,1) # number of cells in each direction
domain = (1e5, 1e5, 1e5) # cm
lowlimfit = 50 # data point to start fit to compute omega_im
uplimfit  = 150 # data point to end fit to compute omega_im

directory_path = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_att_1e-5'
ncell = (1,1,1) # number of cells in each direction
domain = (1e5, 1e5, 1e5) # cm
lowlimfit = 50 # data point to start fit to compute omega_im
uplimfit  = 140 # data point to end fit to compute omega_im

# Calculate the volume of a single cell and the total volume
volume_cell = (domain[0] / ncell[0]) * (domain[1] / ncell[1]) * (domain[2] / ncell[2]) # cm^3
volume = volume_cell * (ncell[0] * ncell[1] * ncell[2]) # cm^3

'''
# Load the list of .plt files
plt_files_list = glob.glob(directory_path + '/plt*.h5')
plt_files_list.sort(key=lambda x: int(x.split('plt')[1].split('.h5')[0]))
print(f"Number of files: {len(plt_files_list)}")
print(f"First plt file: {plt_files_list[0]}")
print(f"Last plt file: {plt_files_list[-1]}")
'''

### Compute and plot average flux densities from mesh data

In [ ]:
all_data_path = os.path.join(directory_path, 'allData.h5')
with h5py.File(all_data_path, 'r') as all_data_file:

    Fx00_Re =  np.array(all_data_file['/Fx00_Re(1|ccm)'][:])
    Fy00_Re =  np.array(all_data_file['/Fy00_Re(1|ccm)'][:])
    Fz00_Re =  np.array(all_data_file['/Fz00_Re(1|ccm)'][:])
    F00_inv_ccm = np.sqrt(Fx00_Re**2+Fy00_Re**2+Fz00_Re**2)
    F00_ave_inv_ccm = np.mean(F00_inv_ccm, axis=tuple(range(1, F00_inv_ccm.ndim)))

    Fx11_Re =  np.array(all_data_file['/Fx11_Re(1|ccm)'][:])
    Fy11_Re =  np.array(all_data_file['/Fy11_Re(1|ccm)'][:])
    Fz11_Re =  np.array(all_data_file['/Fz11_Re(1|ccm)'][:])
    F11_inv_ccm = np.sqrt(Fx11_Re**2 + Fy11_Re**2 + Fz11_Re**2)
    F11_ave_inv_ccm = np.mean(F11_inv_ccm, axis=tuple(range(1, F11_inv_ccm.ndim)))

    Fx22_Re =  np.array(all_data_file['/Fx22_Re(1|ccm)'][:])
    Fy22_Re =  np.array(all_data_file['/Fy22_Re(1|ccm)'][:])
    Fz22_Re =  np.array(all_data_file['/Fz22_Re(1|ccm)'][:])
    F22_inv_ccm = np.sqrt(Fx22_Re**2 + Fy22_Re**2 + Fz22_Re**2)
    F22_ave_inv_ccm = np.mean(F22_inv_ccm, axis=tuple(range(1, F22_inv_ccm.ndim)))
    
    Fx00_Rebar =  np.array(all_data_file['/Fx00_Rebar(1|ccm)'][:])
    Fy00_Rebar =  np.array(all_data_file['/Fy00_Rebar(1|ccm)'][:])
    Fz00_Rebar =  np.array(all_data_file['/Fz00_Rebar(1|ccm)'][:])
    F00bar_inv_ccm = np.sqrt(Fx00_Rebar**2 + Fy00_Rebar**2 + Fz00_Rebar**2)
    F00bar_ave_inv_ccm = np.mean(F00bar_inv_ccm, axis=tuple(range(1, F00bar_inv_ccm.ndim)))

    Fx11_Rebar =  np.array(all_data_file['/Fx11_Rebar(1|ccm)'][:])
    Fy11_Rebar =  np.array(all_data_file['/Fy11_Rebar(1|ccm)'][:])
    Fz11_Rebar =  np.array(all_data_file['/Fz11_Rebar(1|ccm)'][:])
    F11bar_inv_ccm = np.sqrt(Fx11_Rebar**2 + Fy11_Rebar**2 + Fz11_Rebar**2)
    F11bar_ave_inv_ccm = np.mean(F11bar_inv_ccm, axis=tuple(range(1, F11bar_inv_ccm.ndim)))

    Fx22_Rebar =  np.array(all_data_file['/Fx22_Rebar(1|ccm)'][:])
    Fy22_Rebar =  np.array(all_data_file['/Fy22_Rebar(1|ccm)'][:])
    Fz22_Rebar =  np.array(all_data_file['/Fz22_Rebar(1|ccm)'][:])
    F22bar_inv_ccm = np.sqrt(Fx22_Rebar**2 + Fy22_Rebar**2 + Fz22_Rebar**2)
    F22bar_ave_inv_ccm = np.mean(F22bar_inv_ccm, axis=tuple(range(1, F22bar_inv_ccm.ndim)))    

    Fx01_Im =  np.array(all_data_file['/Fx01_Im(1|ccm)'][:])
    Fx01_Re =  np.array(all_data_file['/Fx01_Re(1|ccm)'][:])
    Fy01_Im =  np.array(all_data_file['/Fy01_Im(1|ccm)'][:])
    Fy01_Re =  np.array(all_data_file['/Fy01_Re(1|ccm)'][:])
    Fz01_Im =  np.array(all_data_file['/Fz01_Im(1|ccm)'][:])
    Fz01_Re =  np.array(all_data_file['/Fz01_Re(1|ccm)'][:])
    F01_inv_ccm = np.sqrt( 
        Fx01_Im**2 + Fx01_Re**2 + 
        Fy01_Im**2 + Fy01_Re**2 + 
        Fz01_Im**2 + Fz01_Re**2
    )

    Fx02_Im =  np.array(all_data_file['/Fx02_Im(1|ccm)'][:])
    Fx02_Re =  np.array(all_data_file['/Fx02_Re(1|ccm)'][:])
    Fy02_Im =  np.array(all_data_file['/Fy02_Im(1|ccm)'][:])
    Fy02_Re =  np.array(all_data_file['/Fy02_Re(1|ccm)'][:])
    Fz02_Im =  np.array(all_data_file['/Fz02_Im(1|ccm)'][:])
    Fz02_Re =  np.array(all_data_file['/Fz02_Re(1|ccm)'][:])
    F02_inv_ccm = np.sqrt(
        Fx02_Im**2 + Fx02_Re**2 +
        Fy02_Im**2 + Fy02_Re**2 +
        Fz02_Im**2 + Fz02_Re**2
    )

    Fx12_Im =  np.array(all_data_file['/Fx12_Im(1|ccm)'][:])
    Fx12_Re =  np.array(all_data_file['/Fx12_Re(1|ccm)'][:])
    Fy12_Im =  np.array(all_data_file['/Fy12_Im(1|ccm)'][:])
    Fy12_Re =  np.array(all_data_file['/Fy12_Re(1|ccm)'][:])
    Fz12_Im =  np.array(all_data_file['/Fz12_Im(1|ccm)'][:])
    Fz12_Re =  np.array(all_data_file['/Fz12_Re(1|ccm)'][:])
    F12_inv_ccm = np.sqrt(
        Fx12_Im**2 + Fx12_Re**2 +
        Fy12_Im**2 + Fy12_Re**2 +
        Fz12_Im**2 + Fz12_Re**2
    )

    F01_ave_inv_ccm = np.mean(F01_inv_ccm, axis=tuple(range(1, F01_inv_ccm.ndim)))
    F02_ave_inv_ccm = np.mean(F02_inv_ccm, axis=tuple(range(1, F02_inv_ccm.ndim)))
    F12_ave_inv_ccm = np.mean(F12_inv_ccm, axis=tuple(range(1, F12_inv_ccm.ndim)))

    Fx01_Imbar =  np.array(all_data_file['/Fx01_Imbar(1|ccm)'][:])
    Fx01_Rebar =  np.array(all_data_file['/Fx01_Rebar(1|ccm)'][:])
    Fy01_Imbar =  np.array(all_data_file['/Fy01_Imbar(1|ccm)'][:])
    Fy01_Rebar =  np.array(all_data_file['/Fy01_Rebar(1|ccm)'][:])
    Fz01_Imbar =  np.array(all_data_file['/Fz01_Imbar(1|ccm)'][:])
    Fz01_Rebar =  np.array(all_data_file['/Fz01_Rebar(1|ccm)'][:])
    F01bar_inv_ccm = np.sqrt( 
        Fx01_Imbar**2 + Fx01_Rebar**2 + 
        Fy01_Imbar**2 + Fy01_Rebar**2 + 
        Fz01_Imbar**2 + Fz01_Rebar**2
    )

    Fx02_Imbar =  np.array(all_data_file['/Fx02_Imbar(1|ccm)'][:])
    Fx02_Rebar =  np.array(all_data_file['/Fx02_Rebar(1|ccm)'][:])
    Fy02_Imbar =  np.array(all_data_file['/Fy02_Imbar(1|ccm)'][:])
    Fy02_Rebar =  np.array(all_data_file['/Fy02_Rebar(1|ccm)'][:])
    Fz02_Imbar =  np.array(all_data_file['/Fz02_Imbar(1|ccm)'][:])
    Fz02_Rebar =  np.array(all_data_file['/Fz02_Rebar(1|ccm)'][:])
    F02bar_inv_ccm = np.sqrt(
        Fx02_Imbar**2 + Fx02_Rebar**2 +
        Fy02_Imbar**2 + Fy02_Rebar**2 +
        Fz02_Imbar**2 + Fz02_Rebar**2
    )

    Fx12_Imbar =  np.array(all_data_file['/Fx12_Imbar(1|ccm)'][:])
    Fx12_Rebar =  np.array(all_data_file['/Fx12_Rebar(1|ccm)'][:])
    Fy12_Imbar =  np.array(all_data_file['/Fy12_Imbar(1|ccm)'][:])
    Fy12_Rebar =  np.array(all_data_file['/Fy12_Rebar(1|ccm)'][:])
    Fz12_Imbar =  np.array(all_data_file['/Fz12_Imbar(1|ccm)'][:])
    Fz12_Rebar =  np.array(all_data_file['/Fz12_Rebar(1|ccm)'][:])
    F12bar_inv_ccm = np.sqrt(
        Fx12_Imbar**2 + Fx12_Rebar**2 +
        Fy12_Imbar**2 + Fy12_Rebar**2 +
        Fz12_Imbar**2 + Fz12_Rebar**2
    )

    F01bar_ave_inv_ccm = np.mean(F01_inv_ccm, axis=tuple(range(1, F01bar_inv_ccm.ndim)))
    F02bar_ave_inv_ccm = np.mean(F02_inv_ccm, axis=tuple(range(1, F02bar_inv_ccm.ndim)))
    F12bar_ave_inv_ccm = np.mean(F12_inv_ccm, axis=tuple(range(1, F12bar_inv_ccm.ndim)))

    time_s = np.array(all_data_file['/t(s)'][:])

### Saving data for plotting scripts

In [ ]:
###############################################################################################
# Saving data for plots in HDF5 format

with h5py.File(f"plots/{directory_path.split('/')[-1]}_flux_average_time_evolution.h5", "w") as hf:
    hf.create_dataset("time_s", data=time_s)
    hf.create_dataset("F00_ave_inv_ccm", data=F00_ave_inv_ccm)
    hf.create_dataset("F11_ave_inv_ccm", data=F11_ave_inv_ccm)
    hf.create_dataset("F22_ave_inv_ccm", data=F22_ave_inv_ccm)
    hf.create_dataset("F00bar_ave_inv_ccm", data=F00bar_ave_inv_ccm)
    hf.create_dataset("F11bar_ave_inv_ccm", data=F11bar_ave_inv_ccm)
    hf.create_dataset("F22bar_ave_inv_ccm", data=F22bar_ave_inv_ccm)
    hf.create_dataset("F01_ave_inv_ccm", data=F01_ave_inv_ccm)
    hf.create_dataset("F02_ave_inv_ccm", data=F02_ave_inv_ccm)
    hf.create_dataset("F12_ave_inv_ccm", data=F12_ave_inv_ccm)
    hf.create_dataset("F01bar_ave_inv_ccm", data=F01bar_ave_inv_ccm)
    hf.create_dataset("F02bar_ave_inv_ccm", data=F02bar_ave_inv_ccm)
    hf.create_dataset("F12bar_ave_inv_ccm", data=F12bar_ave_inv_ccm)

### Plot neutrino off diagonal flux number density magnitude

In [ ]:
###################################################################################
# PLOT DIAGONALS AVERAGE NUMBER DENSITIES

fig, ax = plt.subplots(figsize=(10, 6))

a=None

ax.plot(time_s[0:a], F01_ave_inv_ccm[0:a], label=r'$<\,|\,f_{e\mu}\,|\,>$', linewidth=3)
ax.plot(time_s[0:a], F02_ave_inv_ccm[0:a], label=r'$<\,|\,f_{e\tau}\,|\,>$', linewidth=3)
ax.plot(time_s[0:a], F12_ave_inv_ccm[0:a], label=r'$<\,|\,f_{\mu\tau}\,|\,>$', linewidth=3)

ax.set_xlabel(r'$t \, [\mathrm{s}]$')
ax.set_ylabel(r'$\langle |\,f\,| \rangle \; [\mathrm{cm}^{-3}]$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=20)
pf.apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig(f"plots/{directory_path.split('/')[-1]}_f_average_off_diagonal_meshdata.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

In [ ]:
# ####################################################
# # COMPUTE SLOPE OF THE AVERAGE NUMBER DENSITY
# ####################################################

def fit_exponential(tiempo, funcion):
    b, ln_a = np.polyfit(tiempo, np.log(funcion), deg=1)
    a = np.exp(ln_a)
    return a, b

In [ ]:
################# neutrinos #################
fig, ax = plt.subplots(figsize=(10, 6))
# e mu 
amplitude_eu, omega_eu = fit_exponential(time_s[lowlimfit:uplimfit], F01_ave_inv_ccm[lowlimfit:uplimfit])
print(f"amplitude_eu = {amplitude_eu:.3e}\nomega_eu = {omega_eu:.3e}")
ax.plot(time_s, F01_ave_inv_ccm, color='C0', linewidth=3)
ax.plot(time_s[lowlimfit:uplimfit], amplitude_eu * np.exp(omega_eu * time_s[lowlimfit:uplimfit]), linestyle='dashed',
    label=fr'$\mathrm{{Im}}\,\omega_{{e\mu}}={omega_eu:.3e}$', linewidth=3, color='C1')

# e tau
amplitude_et, omega_et = fit_exponential(time_s[lowlimfit:uplimfit], F02_ave_inv_ccm[lowlimfit:uplimfit])
print(f"amplitude_et = {amplitude_et:.3e}\nomega_et = {omega_et:.3e}")
ax.plot(time_s, F02_ave_inv_ccm, color='C2', linewidth=3)
ax.plot(time_s[lowlimfit:uplimfit], amplitude_et * np.exp(omega_et * time_s[lowlimfit:uplimfit]), linestyle='dashed',
    label=fr'$\mathrm{{Im}}\,\omega_{{e\tau}}={omega_et:.3e}$', linewidth=3, color='C3')

# mu tau
amplitude_ut, omega_ut = fit_exponential(time_s[lowlimfit:uplimfit], F12_ave_inv_ccm[lowlimfit:uplimfit])
print(f"amplitude_ut = {amplitude_ut:.3e}\nomega_ut = {omega_ut:.3e}")
ax.plot(time_s, F12_ave_inv_ccm, color='C4', linewidth=3)
ax.plot(time_s[lowlimfit:uplimfit], amplitude_ut * np.exp(omega_ut * time_s[lowlimfit:uplimfit]), linestyle='dashed',
    label=fr'$\mathrm{{Im}}\,\omega_{{\mu\tau}}={omega_ut:.3e}$', linewidth=3, color='C5')

ax.set_xlabel(r'$t\;[\mathrm{s}]$')
ax.set_ylabel(r'$\langle |\,f\,| \rangle\;[\mathrm{cm}^{-3}]$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=20)
# ax.set_xlim(0,3e-8)
pf.apply_custom_settings(ax, leg, log_scale_y=True)
plt.savefig(f"plots/{directory_path.split('/')[-1]}_n_average_offdiagonal_meshdata_withslope.pdf", bbox_inches='tight')
plt.tight_layout()
plt.show()
plt.close(fig)

In [ ]:
# ################# neutrinos #################
fig, ax = plt.subplots(figsize=(10, 6))

# ee
amplitude_ee, omega_ee = fit_exponential(
    time_s[lowlimfit:uplimfit], F00_ave_inv_ccm[lowlimfit:uplimfit]
)
print(f"amplitude_ee = {amplitude_ee:.3e}\nomega_ee = {omega_ee:.3e}")
ax.plot(time_s, F00_ave_inv_ccm, color='C0', linewidth=3)
ax.plot(
    time_s[lowlimfit:uplimfit],
    amplitude_ee * np.exp(omega_ee * time_s[lowlimfit:uplimfit]),
    linestyle='dashed',
    label=fr'$\mathrm{{Im}}\,\omega_{{ee}}={omega_ee:.3e}$',
    linewidth=3,
    color='C1'
)

# uu
amplitude_uu, omega_uu = fit_exponential(
    time_s[lowlimfit:uplimfit], F11_ave_inv_ccm[lowlimfit:uplimfit]
)
print(f"amplitude_uu = {amplitude_uu:.3e}\nomega_uu = {omega_uu:.3e}")
ax.plot(time_s, F11_ave_inv_ccm, color='C2', linewidth=3)
ax.plot(
    time_s[lowlimfit:uplimfit],
    amplitude_uu * np.exp(omega_uu * time_s[lowlimfit:uplimfit]),
    linestyle='dashed',
    label=fr'$\mathrm{{Im}}\,\omega_{{\mu\mu}}={omega_uu:.3e}$',
    linewidth=3,
    color='C3'
)

# tt
amplitude_tt, omega_tt = fit_exponential(
    time_s[lowlimfit:uplimfit], F22_ave_inv_ccm[lowlimfit:uplimfit]
)
print(f"amplitude_tt = {amplitude_tt:.3e}\nomega_tt = {omega_tt:.3e}")
ax.plot(time_s, F22_ave_inv_ccm, color='C4', linewidth=3)
ax.plot(
    time_s[lowlimfit:uplimfit],
    amplitude_tt * np.exp(omega_tt * time_s[lowlimfit:uplimfit]),
    linestyle='dashed',
    label=fr'$\mathrm{{Im}}\,\omega_{{\tau\tau}}={omega_tt:.3e}$',
    linewidth=3,
    color='C5'
)

ax.set_xlabel(r'$t\;[\mathrm{s}]$')
ax.set_ylabel(r'$\langle |\,f\,| \rangle\;[\mathrm{cm}^{-3}]$')
leg = ax.legend(framealpha=0.0, ncol=1, fontsize=20)
pf.apply_custom_settings(ax, leg, log_scale_y=True)

plt.savefig(
    f"plots/{directory_path.split('/')[-1]}_n_average_offdiagonal_meshdata_withslope.pdf",
    bbox_inches='tight'
)
plt.tight_layout()
plt.show()
plt.close(fig)